# 🔬 Google Embedding Models Benchmark

## Setup & Imports

In [1]:
import time
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from google import genai
from google.genai import types
from google.oauth2 import service_account
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports successful")

✅ Imports successful


## Load Credentials

In [3]:
# Load Credentials

# Load environment variables
import os  # <-- Add this line
env_path = Path('../vivah_api/.env')
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded .env from: {env_path.absolute()}")
else:
    load_dotenv()
    print("⚠️  Using default .env")

# Get credentials path
CREDENTIALS_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if CREDENTIALS_PATH and not os.path.isabs(CREDENTIALS_PATH):
    CREDENTIALS_PATH = str((Path('../vivah_api') / CREDENTIALS_PATH).absolute())

PROJECT_ID = os.getenv("GCP_PROJECT_ID", "triple-name-473801-t7")
LOCATION = os.getenv("GCP_LOCATION", "us-central1")

print(f"\n📋 Configuration:")
print(f"   Project: {PROJECT_ID}")
print(f"   Location: {LOCATION}")
print(f"   Credentials: {CREDENTIALS_PATH}")
print(f"   Exists: {os.path.exists(CREDENTIALS_PATH) if CREDENTIALS_PATH else False}")

✅ Loaded .env from: f:\Vivahai\Vivahai_backend_v2\research\..\vivah_api\.env

📋 Configuration:
   Project: triple-name-473801-t7
   Location: us-central1
   Credentials: f:\Vivahai\Vivahai_backend_v2\research\..\vivah_api\service-account.json
   Exists: True


## Initialize Google GenAI Client

In [4]:
# Initialize Google GenAI Client

# Create credentials
SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]
credentials = service_account.Credentials.from_service_account_file(
    CREDENTIALS_PATH, scopes=SCOPES
)

# Initialize GenAI client
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
    credentials=credentials,
)

print("✅ Google GenAI client initialized successfully!")

✅ Google GenAI client initialized successfully!


## Test Configuration

In [6]:
# Test Configuration

# Models to test
MODELS = [
    "text-embedding-004",
    "gemini-embedding-001",
    "text-embedding-005",
    "text-multilingual-embedding-002"
]

# Dimensions to test
DIMENSIONS = [768, 1152, 1536, 3072]

# Test text
TEST_TEXT = "Looking for a life partner who is educated, kind and family oriented."

print(f"\n📊 Test Configuration:")
print(f"   Models: {len(MODELS)}")
print(f"   Dimensions: {DIMENSIONS}")
print(f"   Total tests: {len(MODELS) * len(DIMENSIONS)}")
print(f"   Test text: '{TEST_TEXT}'")


📊 Test Configuration:
   Models: 4
   Dimensions: [768, 1152, 1536, 3072]
   Total tests: 16
   Test text: 'Looking for a life partner who is educated, kind and family oriented.'


## Benchmark Function

In [7]:
# Benchmark Function

def test_embedding(
    model_name: str, 
    dimension: int, 
    text: str = TEST_TEXT
) -> Dict:
    """
    Test embedding generation for a specific model and dimension.
    
    Returns:
        Dict with test results
    """
    result = {
        'model_name': model_name,
        'requested_dim': dimension,
        'actual_dim': None,
        'status': 'FAILED',
        'latency_ms': None,
        'remarks': ''
    }
    
    try:
        # Start timer
        start_time = time.time()
        
        # Generate embedding
        response = client.models.embed_content(
            model=model_name,
            contents=text,
            config=types.EmbedContentConfig(
                task_type="RETRIEVAL_DOCUMENT",
                output_dimensionality=dimension
            )
        )
        
        # Calculate latency
        latency = (time.time() - start_time) * 1000  # Convert to ms
        
        # Get embedding vector
        embedding = response.embeddings[0].values
        actual_dim = len(embedding)
        
        # Update result
        result['actual_dim'] = actual_dim
        result['latency_ms'] = round(latency, 2)
        result['status'] = 'SUCCESS'
        
        # Check if dimension matches
        if actual_dim == dimension:
            result['remarks'] = '✅ Dimension matches'
        else:
            result['remarks'] = f'⚠️ Dimension mismatch: requested {dimension}, got {actual_dim}'
            
    except Exception as e:
        error_msg = str(e)
        result['remarks'] = f'❌ Error: {error_msg[:100]}'
        
        # Categorize error
        if 'not found' in error_msg.lower():
            result['remarks'] = '❌ Model not found'
        elif 'dimension' in error_msg.lower():
            result['remarks'] = '❌ Unsupported dimension'
        elif 'quota' in error_msg.lower():
            result['remarks'] = '❌ Quota exceeded'
        elif 'permission' in error_msg.lower():
            result['remarks'] = '❌ Permission denied'
    
    return result

print("✅ Benchmark function defined")

✅ Benchmark function defined


## Run Benchmark Tests

In [8]:
# Run Benchmark Tests

print("\n🚀 Starting benchmark tests...\n")
print("="*80)

results = []
total_tests = len(MODELS) * len(DIMENSIONS)
current_test = 0

for model in MODELS:
    print(f"\n📦 Testing model: {model}")
    print("-" * 80)
    
    for dim in DIMENSIONS:
        current_test += 1
        print(f"   [{current_test}/{total_tests}] Dimension: {dim}...", end=" ")
        
        # Run test
        result = test_embedding(model, dim)
        results.append(result)
        
        # Print result
        if result['status'] == 'SUCCESS':
            print(f"✅ {result['actual_dim']}D in {result['latency_ms']}ms")
        else:
            print(f"❌ {result['remarks']}")
        
        # Rate limiting (avoid quota issues)
        time.sleep(0.5)

print("\n" + "="*80)
print("✅ Benchmark complete!\n")


🚀 Starting benchmark tests...


📦 Testing model: text-embedding-004
--------------------------------------------------------------------------------
   [1/16] Dimension: 768... ✅ 768D in 3067.17ms
   [2/16] Dimension: 1152... ❌ ❌ Unsupported dimension
   [3/16] Dimension: 1536... ❌ ❌ Unsupported dimension
   [4/16] Dimension: 3072... ❌ ❌ Unsupported dimension

📦 Testing model: gemini-embedding-001
--------------------------------------------------------------------------------
   [5/16] Dimension: 768... ✅ 768D in 726.98ms
   [6/16] Dimension: 1152... ✅ 1152D in 496.51ms
   [7/16] Dimension: 1536... ✅ 1536D in 740.49ms
   [8/16] Dimension: 3072... ✅ 3072D in 710.89ms

📦 Testing model: text-embedding-005
--------------------------------------------------------------------------------
   [9/16] Dimension: 768... ✅ 768D in 416.58ms
   [10/16] Dimension: 1152... ❌ ❌ Unsupported dimension
   [11/16] Dimension: 1536... ❌ ❌ Unsupported dimension
   [12/16] Dimension: 3072... ❌ ❌ Unsupported 

## Results Table

In [10]:
# Results Table

# Create DataFrame
df_results = pd.DataFrame(results)

# Display full table
print("📊 COMPLETE RESULTS TABLE")
print("="*80)
print(df_results.to_string())

# Save to CSV
csv_path = 'embedding_benchmark_results.csv'
df_results.to_csv(csv_path, index=False)
print(f"\n💾 Results saved to: {csv_path}")

📊 COMPLETE RESULTS TABLE
                         model_name  requested_dim  actual_dim   status  latency_ms                  remarks
0                text-embedding-004            768       768.0  SUCCESS     3067.17      ✅ Dimension matches
1                text-embedding-004           1152         NaN   FAILED         NaN  ❌ Unsupported dimension
2                text-embedding-004           1536         NaN   FAILED         NaN  ❌ Unsupported dimension
3                text-embedding-004           3072         NaN   FAILED         NaN  ❌ Unsupported dimension
4              gemini-embedding-001            768       768.0  SUCCESS      726.98      ✅ Dimension matches
5              gemini-embedding-001           1152      1152.0  SUCCESS      496.51      ✅ Dimension matches
6              gemini-embedding-001           1536      1536.0  SUCCESS      740.49      ✅ Dimension matches
7              gemini-embedding-001           3072      3072.0  SUCCESS      710.89      ✅ Dimension ma

## Analysis: Success Rate by Model

In [11]:
# Analysis: Success Rate by Model

print("\n\n📈 SUCCESS RATE BY MODEL")
print("="*80)

for model in MODELS:
    model_results = df_results[df_results['model_name'] == model]
    success_count = len(model_results[model_results['status'] == 'SUCCESS'])
    total_count = len(model_results)
    success_rate = (success_count / total_count) * 100
    
    print(f"\n{model}:")
    print(f"   Success: {success_count}/{total_count} ({success_rate:.1f}%)")
    
    # Show supported dimensions
    successful = model_results[model_results['status'] == 'SUCCESS']
    if len(successful) > 0:
        supported_dims = successful['actual_dim'].tolist()
        print(f"   Supported dimensions: {supported_dims}")
        
        # Show latency stats
        avg_latency = successful['latency_ms'].mean()
        min_latency = successful['latency_ms'].min()
        max_latency = successful['latency_ms'].max()
        print(f"   Latency: avg={avg_latency:.2f}ms, min={min_latency:.2f}ms, max={max_latency:.2f}ms")
    else:
        print(f"   ❌ No successful tests")



📈 SUCCESS RATE BY MODEL

text-embedding-004:
   Success: 1/4 (25.0%)
   Supported dimensions: [768.0]
   Latency: avg=3067.17ms, min=3067.17ms, max=3067.17ms

gemini-embedding-001:
   Success: 4/4 (100.0%)
   Supported dimensions: [768.0, 1152.0, 1536.0, 3072.0]
   Latency: avg=668.72ms, min=496.51ms, max=740.49ms

text-embedding-005:
   Success: 1/4 (25.0%)
   Supported dimensions: [768.0]
   Latency: avg=416.58ms, min=416.58ms, max=416.58ms

text-multilingual-embedding-002:
   Success: 1/4 (25.0%)
   Supported dimensions: [768.0]
   Latency: avg=475.06ms, min=475.06ms, max=475.06ms


## Analysis: Success Rate by Dimension

In [12]:
# Analysis: Success Rate by Dimension

print("\n\n📊 SUCCESS RATE BY DIMENSION")
print("="*80)

for dim in DIMENSIONS:
    dim_results = df_results[df_results['requested_dim'] == dim]
    success_count = len(dim_results[dim_results['status'] == 'SUCCESS'])
    total_count = len(dim_results)
    success_rate = (success_count / total_count) * 100
    
    print(f"\n{dim} dimensions:")
    print(f"   Success: {success_count}/{total_count} ({success_rate:.1f}%)")
    
    # Show which models support this dimension
    successful = dim_results[dim_results['status'] == 'SUCCESS']
    if len(successful) > 0:
        supported_models = successful['model_name'].tolist()
        print(f"   Supported by: {', '.join(supported_models)}")
    else:
        print(f"   ❌ Not supported by any model")



📊 SUCCESS RATE BY DIMENSION

768 dimensions:
   Success: 4/4 (100.0%)
   Supported by: text-embedding-004, gemini-embedding-001, text-embedding-005, text-multilingual-embedding-002

1152 dimensions:
   Success: 1/4 (25.0%)
   Supported by: gemini-embedding-001

1536 dimensions:
   Success: 1/4 (25.0%)
   Supported by: gemini-embedding-001

3072 dimensions:
   Success: 1/4 (25.0%)
   Supported by: gemini-embedding-001


## Analysis: Latency Comparison

In [13]:
# Analysis: Latency Comparison

print("\n\n⚡ LATENCY COMPARISON")
print("="*80)

# Filter successful tests only
successful_tests = df_results[df_results['status'] == 'SUCCESS'].copy()

if len(successful_tests) > 0:
    # Pivot table for better visualization
    latency_pivot = successful_tests.pivot_table(
        values='latency_ms',
        index='model_name',
        columns='requested_dim',
        aggfunc='mean'
    )
    
    print("\nAverage Latency (ms) by Model and Dimension:")
    print(latency_pivot.to_string())
    
    # Find fastest model
    avg_latency_by_model = successful_tests.groupby('model_name')['latency_ms'].mean().sort_values()
    print(f"\n🏆 Fastest model overall: {avg_latency_by_model.index[0]} ({avg_latency_by_model.iloc[0]:.2f}ms avg)")
    
    # Find fastest dimension
    avg_latency_by_dim = successful_tests.groupby('requested_dim')['latency_ms'].mean().sort_values()
    print(f"🏆 Fastest dimension: {avg_latency_by_dim.index[0]}D ({avg_latency_by_dim.iloc[0]:.2f}ms avg)")
else:
    print("❌ No successful tests to analyze latency")



⚡ LATENCY COMPARISON

Average Latency (ms) by Model and Dimension:
requested_dim                       768     1152    1536    3072
model_name                                                      
gemini-embedding-001              726.98  496.51  740.49  710.89
text-embedding-004               3067.17     NaN     NaN     NaN
text-embedding-005                416.58     NaN     NaN     NaN
text-multilingual-embedding-002   475.06     NaN     NaN     NaN

🏆 Fastest model overall: text-embedding-005 (416.58ms avg)
🏆 Fastest dimension: 1152D (496.51ms avg)


## Maximum Supported Dimensions

In [14]:
# Maximum Supported Dimensions

print("\n\n📏 MAXIMUM SUPPORTED DIMENSIONS")
print("="*80)

for model in MODELS:
    model_results = df_results[
        (df_results['model_name'] == model) & 
        (df_results['status'] == 'SUCCESS')
    ]
    
    if len(model_results) > 0:
        max_dim = model_results['actual_dim'].max()
        print(f"\n{model}:")
        print(f"   Max dimension: {max_dim}")
        print(f"   All supported: {sorted(model_results['actual_dim'].unique())}")
    else:
        print(f"\n{model}:")
        print(f"   ❌ No successful tests")



📏 MAXIMUM SUPPORTED DIMENSIONS

text-embedding-004:
   Max dimension: 768.0
   All supported: [np.float64(768.0)]

gemini-embedding-001:
   Max dimension: 3072.0
   All supported: [np.float64(768.0), np.float64(1152.0), np.float64(1536.0), np.float64(3072.0)]

text-embedding-005:
   Max dimension: 768.0
   All supported: [np.float64(768.0)]

text-multilingual-embedding-002:
   Max dimension: 768.0
   All supported: [np.float64(768.0)]


## 📋 CONCLUSIONS & RECOMMENDATIONS

In [15]:
# CONCLUSIONS & RECOMMENDATIONS

print("\n\n" + "="*80)
print("📋 FINAL CONCLUSIONS & RECOMMENDATIONS")
print("="*80)

# 1. Models with dimension control
print("\n1️⃣ MODELS WITH CONFIGURABLE DIMENSIONS:")
print("-" * 80)
for model in MODELS:
    model_results = df_results[
        (df_results['model_name'] == model) & 
        (df_results['status'] == 'SUCCESS')
    ]
    if len(model_results) > 0:
        dims = sorted(model_results['actual_dim'].unique())
        print(f"✅ {model}")
        print(f"   Supports: {dims}")
    else:
        print(f"❌ {model}")
        print(f"   No dimension control or model unavailable")

# 2. Maximum dimensions
print("\n2️⃣ MAXIMUM SUPPORTED DIMENSIONS:")
print("-" * 80)
successful_models = df_results[df_results['status'] == 'SUCCESS']['model_name'].unique()
if len(successful_models) > 0:
    max_overall = df_results[df_results['status'] == 'SUCCESS']['actual_dim'].max()
    print(f"Highest dimension tested successfully: {max_overall}")
    
    models_with_max = df_results[
        (df_results['status'] == 'SUCCESS') & 
        (df_results['actual_dim'] == max_overall)
    ]['model_name'].unique()
    print(f"Models supporting {max_overall}D: {', '.join(models_with_max)}")
else:
    print("❌ No successful tests")

# 3. Production recommendations
print("\n3️⃣ PRODUCTION RECOMMENDATIONS:")
print("-" * 80)

if len(successful_tests) > 0:
    # Find most reliable dimension (highest success rate)
    dim_success_rate = df_results.groupby('requested_dim').apply(
        lambda x: (x['status'] == 'SUCCESS').sum() / len(x) * 100
    ).sort_values(ascending=False)
    
    best_dim = dim_success_rate.index[0]
    best_dim_rate = dim_success_rate.iloc[0]
    
    print(f"\n🎯 Most Reliable Dimension: {best_dim}")
    print(f"   Success rate: {best_dim_rate:.1f}%")
    print(f"   Supported by {len(df_results[(df_results['requested_dim'] == best_dim) & (df_results['status'] == 'SUCCESS')])} models")
    
    # Find best model (highest success rate + lowest latency)
    model_stats = successful_tests.groupby('model_name').agg({
        'latency_ms': 'mean',
        'status': 'count'
    }).rename(columns={'status': 'success_count'})
    
    if len(model_stats) > 0:
        best_model = model_stats['latency_ms'].idxmin()
        best_latency = model_stats.loc[best_model, 'latency_ms']
        
        print(f"\n🏆 Recommended Model: {best_model}")
        print(f"   Average latency: {best_latency:.2f}ms")
        print(f"   Successful tests: {model_stats.loc[best_model, 'success_count']}")
    
    # Specific use case recommendations
    print("\n💡 USE CASE RECOMMENDATIONS:")
    print("-" * 80)
    
    # Speed priority
    if len(successful_tests) > 0:
        fastest = successful_tests.loc[successful_tests['latency_ms'].idxmin()]
        print(f"\n⚡ For Speed (lowest latency):")
        print(f"   Model: {fastest['model_name']}")
        print(f"   Dimension: {fastest['actual_dim']}")
        print(f"   Latency: {fastest['latency_ms']:.2f}ms")
    
    # Accuracy priority (highest dimension)
    highest_dim_tests = successful_tests[successful_tests['actual_dim'] == successful_tests['actual_dim'].max()]
    if len(highest_dim_tests) > 0:
        best_high_dim = highest_dim_tests.loc[highest_dim_tests['latency_ms'].idxmin()]
        print(f"\n🎯 For Accuracy (highest dimension):")
        print(f"   Model: {best_high_dim['model_name']}")
        print(f"   Dimension: {best_high_dim['actual_dim']}")
        print(f"   Latency: {best_high_dim['latency_ms']:.2f}ms")
    
    # Balanced
    mid_dim_tests = successful_tests[successful_tests['actual_dim'] == 768]
    if len(mid_dim_tests) > 0:
        best_balanced = mid_dim_tests.loc[mid_dim_tests['latency_ms'].idxmin()]
        print(f"\n⚖️ For Balance (768D - speed + accuracy):")
        print(f"   Model: {best_balanced['model_name']}")
        print(f"   Dimension: {best_balanced['actual_dim']}")
        print(f"   Latency: {best_balanced['latency_ms']:.2f}ms")

else:
    print("❌ No successful tests to provide recommendations")

print("\n" + "="*80)
print("✅ BENCHMARK ANALYSIS COMPLETE")
print("="*80)



📋 FINAL CONCLUSIONS & RECOMMENDATIONS

1️⃣ MODELS WITH CONFIGURABLE DIMENSIONS:
--------------------------------------------------------------------------------
✅ text-embedding-004
   Supports: [np.float64(768.0)]
✅ gemini-embedding-001
   Supports: [np.float64(768.0), np.float64(1152.0), np.float64(1536.0), np.float64(3072.0)]
✅ text-embedding-005
   Supports: [np.float64(768.0)]
✅ text-multilingual-embedding-002
   Supports: [np.float64(768.0)]

2️⃣ MAXIMUM SUPPORTED DIMENSIONS:
--------------------------------------------------------------------------------
Highest dimension tested successfully: 3072.0
Models supporting 3072.0D: gemini-embedding-001

3️⃣ PRODUCTION RECOMMENDATIONS:
--------------------------------------------------------------------------------

🎯 Most Reliable Dimension: 768
   Success rate: 100.0%
   Supported by 4 models

🏆 Recommended Model: text-embedding-005
   Average latency: 416.58ms
   Successful tests: 1

💡 USE CASE RECOMMENDATIONS:
-------------------

## Summary Statistics

In [16]:
# Summary Statistics

print("\n\n📊 SUMMARY STATISTICS")
print("="*80)

total_tests = len(df_results)
successful_tests_count = len(df_results[df_results['status'] == 'SUCCESS'])
failed_tests_count = len(df_results[df_results['status'] == 'FAILED'])

print(f"\nTotal tests run: {total_tests}")
print(f"Successful: {successful_tests_count} ({successful_tests_count/total_tests*100:.1f}%)")
print(f"Failed: {failed_tests_count} ({failed_tests_count/total_tests*100:.1f}%)")

if successful_tests_count > 0:
    print(f"\nLatency statistics (successful tests):")
    print(f"   Mean: {df_results[df_results['status'] == 'SUCCESS']['latency_ms'].mean():.2f}ms")
    print(f"   Median: {df_results[df_results['status'] == 'SUCCESS']['latency_ms'].median():.2f}ms")
    print(f"   Min: {df_results[df_results['status'] == 'SUCCESS']['latency_ms'].min():.2f}ms")
    print(f"   Max: {df_results[df_results['status'] == 'SUCCESS']['latency_ms'].max():.2f}ms")

print("\n" + "="*80)
print("🎉 BENCHMARK COMPLETE!")
print("="*80)



📊 SUMMARY STATISTICS

Total tests run: 16
Successful: 7 (43.8%)
Failed: 9 (56.2%)

Latency statistics (successful tests):
   Mean: 947.67ms
   Median: 710.89ms
   Min: 416.58ms
   Max: 3067.17ms

🎉 BENCHMARK COMPLETE!
